# EVOSOCBRAIN

### Creating a BIDS style participants.tsv file

**Author: Olaf Borghi**

**Contact: olafborghi@gmail.com**


### Setup

Install and import libraries

In [14]:
# import modules
import os
import numpy as np # math and arrays
import pandas as pd # data frames and data analysis
from scipy import stats # statistical methods
from matplotlib import pyplot as plt # plotting
from pathlib import Path # flexible path handling

Load data

Select a local file, in this case the xlsx file containing the data. I stored the datafile in the same folder as this script, making it easy to access across platforms.

In [15]:
# Fixed paths
work_dir = Path(os.getcwd()) # change to your local working directory (data is stored here as well in my case)
data_dir = Path('/mnt/o/AON2bids/02_participants')

In [16]:
print(data_dir / "data_evobrain.xlsx")

/mnt/o/AON2bids/02_participants/data_evobrain.xlsx


In [17]:
# load the file
dataRaw = pd.read_excel(data_dir / "data_evobrain.xlsx") 
dataRaw.tail()

,pn_code,age,gender,handedness,QCAE_1,QCAE_2,QCAE_3,QCAE_4,QCAE_5,QCAE_6,...,QCAE_31,dog_exp_1,dog_exp_2,dog_exp_3,dog_exp_3_opt1,dog_exp_4,dog_exp_4_opt1,dog_exp_5,dog_exp_5_opt1,dog_exp_5_opt2
35,EV36,23,2,1,2,1,1,2,3,1,...,3,3,3,2,NaN,2,NaN,2,NaN,NaN
36,EV37,25,2,1,2,1,2,1,1,1,...,2,4,3,2,NaN,2,NaN,2,NaN,NaN
37,EV38,23,2,1,2,3,2,2,3,3,...,3,4,2,2,NaN,2,NaN,2,NaN,NaN
38,EV39,27,1,1,3,1,3,1,1,1,...,4,5,4,1,20.0,1,1.0,1,NaN,"Hundeschule 1 Monat, danach selbst weitertrain..."
39,EV40,21,2,1,4,2,1,1,2,1,...,1,4,3,2,NaN,2,NaN,2,NaN,NaN


Recode items and variables

In [18]:
# new df to keep original data
dataAnalysis = dataRaw 

# rename factor levels
dataAnalysis['gender'].replace({1: 0, 2: 1, 3: 2}, inplace=True) # recode to 0 = woman, 1 = man 2 = other
dataAnalysis['dog_exp_3'].replace({1: 1, 2: 0}, inplace=True) # recode to 0 = no and 1 = yes
dataAnalysis['dog_exp_4'].replace({1: 1, 2: 0}, inplace=True) # recode to 0 = no and 1 = yes
dataAnalysis['dog_exp_5'].replace({1: 1, 2: 0}, inplace=True) # recode to 0 = no and 1 = yes

# QCAE items 1, 2, 17 and 29 need to be inverted
dataAnalysis['QCAE_1'] = dataAnalysis['QCAE_1'].map({1:4, 2:3, 3:2, 4:1}) 
dataAnalysis['QCAE_2'] = dataAnalysis['QCAE_2'].map({1:4, 2:3, 3:2, 4:1})
dataAnalysis['QCAE_17'] = dataAnalysis['QCAE_17'].map({1:4, 2:3, 3:2, 4:1})
dataAnalysis['QCAE_29'] = dataAnalysis['QCAE_29'].map({1:4, 2:3, 3:2, 4:1})

dataAnalysis.head()

,pn_code,age,gender,handedness,QCAE_1,QCAE_2,QCAE_3,QCAE_4,QCAE_5,QCAE_6,...,QCAE_31,dog_exp_1,dog_exp_2,dog_exp_3,dog_exp_3_opt1,dog_exp_4,dog_exp_4_opt1,dog_exp_5,dog_exp_5_opt1,dog_exp_5_opt2
0,EV01,19,0,1,2,2,2,1,2,2,...,2,4,3,0,NaN,0,NaN,0,NaN,NaN
1,EV02,21,1,1,1,2,1,2,2,2,...,3,3,2,0,NaN,0,NaN,0,NaN,NaN
2,EV03,26,1,1,1,3,1,1,1,1,...,1,4,3,0,NaN,0,NaN,0,NaN,NaN
3,EV04,20,0,1,2,2,2,2,2,3,...,2,5,4,1,20.0,1,1.0,1,3.0,Hundeschule
4,EV05,20,0,1,1,2,2,1,1,3,...,2,4,3,0,NaN,0,NaN,0,NaN,NaN


Create a new data frame only including the variables relevant for analysis 

In [19]:
# iniitiate dataframe
data = pd.DataFrame()

# add participant_id
data['participant_id'] = [f"sub-{i:02d}" for i in range(1, 41)]
data.head()

,participant_id
0,sub-01
1,sub-02
2,sub-03
3,sub-04
4,sub-05


In [20]:
# keep sex and age variable
data['age'] = dataAnalysis['age']
data['sex'] = dataAnalysis['gender']
data['handedness'] = dataAnalysis['handedness']

# calculate QCAE subscale scores
data['perspective_taking'] = dataAnalysis.iloc[:, [18,19,22,23,24,25,27,28,29,30]].mean(axis=1)
data['online_simulation'] = dataAnalysis.iloc[:, [4,6,7,8,9,21,31,33,34]].mean(axis=1)
data['emotion_contagion'] = dataAnalysis.iloc[:, [11,12,16,17]].mean(axis=1)
data['proximal_responsivity'] = dataAnalysis.iloc[:, [10,13,15,26]].mean(axis=1)
data['peripheral_responsivity'] = dataAnalysis.iloc[:, [5,14,20,32]].mean(axis=1)

# calculate QCAE higher order factors
data['cognitive_empathy'] = data.loc[:, ['perspective_taking','online_simulation']].mean(axis=1)
data['affective_empathy'] = data.loc[:, ['emotion_contagion','proximal_responsivity','peripheral_responsivity']].mean(axis=1)

# dog expertise questionnaire
data['affection'] = dataAnalysis['dog_exp_1']
data['knowledge'] = dataAnalysis['dog_exp_2']
data['shared_household'] = dataAnalysis['dog_exp_3']
data['shared_household_years'] = dataAnalysis['dog_exp_3_opt1']
data['ownership'] = dataAnalysis['dog_exp_4']
data['ownership_count'] = dataAnalysis['dog_exp_4_opt1']
data['expertise'] = dataAnalysis['dog_exp_5']
data['expertise_years'] = dataAnalysis['dog_exp_5_opt1']
data['expertise_open'] = dataAnalysis['dog_exp_5_opt2']

# replace handedness = 1 with R for right-handed
#data.loc[data['handedness'] == 1, 'handedness'] = "R"

# create a new dataframe with the variable "dog_expert" when participants answered with yes either on household, ownership, or expertise
dog_expert = pd.DataFrame()
dog_expert['dog_expert'] = np.arange(0,40) # initiate the variable
for i, sbj in enumerate(data['participant_id']):
    if data['shared_household'][i] == 1 or data['ownership'][i] == 1 or data['expertise'][i] == 1:
        dog_expert['dog_expert'][i] = 1
    else:
        dog_expert['dog_expert'][i] = 0

# insert the dog expertise variable to our og dataframe
data.insert (3, "dog_expert", dog_expert)

# replace missing values with "n/a"
data = data.replace(r'^\s*$', np.nan, regex=True)
  
data

,participant_id,age,sex,dog_expert,handedness,perspective_taking,online_simulation,emotion_contagion,proximal_responsivity,peripheral_responsivity,...,affective_empathy,affection,knowledge,shared_household,shared_household_years,ownership,ownership_count,expertise,expertise_years,expertise_open
0,sub-01,19,0,0,1,1.6,1.888889,1.75,1.50,1.75,...,1.666667,4,3,0,NaN,0,NaN,0,NaN,NaN
1,sub-02,21,1,0,1,2.7,1.666667,2.50,3.00,2.50,...,2.666667,3,2,0,NaN,0,NaN,0,NaN,NaN
2,sub-03,26,1,0,1,1.0,1.111111,2.25,2.00,2.25,...,2.166667,4,3,0,NaN,0,NaN,0,NaN,NaN
3,sub-04,20,0,1,1,2.0,2.222222,2.25,1.75,2.00,...,2.000000,5,4,1,20.0,1,1.0,1,3.0,Hundeschule
4,sub-05,20,0,0,1,1.5,1.666667,1.50,2.00,1.75,...,1.750000,4,3,0,NaN,0,NaN,0,NaN,NaN
5,sub-06,21,0,0,1,1.7,1.444444,2.00,2.00,2.25,...,2.083333,5,3,0,NaN,0,NaN,0,NaN,NaN
6,sub-07,22,1,0,1,1.1,2.444444,2.00,1.50,2.50,...,2.000000,5,3,0,NaN,0,NaN,0,NaN,NaN
7,sub-08,22,0,0,1,1.5,2.000000,2.25,1.75,1.50,...,1.833333,4,3,0,NaN,0,NaN,0,NaN,NaN
8,sub-09,27,0,1,1,2.0,2.444444,1.75,1.25,1.50,...,1.500000,4,3,1,3.0,0,NaN,1,1.0,Familienhund als Welpe trainiert und Begleitun...
9,sub-10,25,0,1,1,2.4,1.888889,2.50,3.00,3.50,...,3.000000,5,3,1,8.0,1,2.0,0,NaN,NaN


### Variable descriptives

In [21]:
data.describe()

,age,sex,dog_expert,handedness,perspective_taking,online_simulation,emotion_contagion,proximal_responsivity,peripheral_responsivity,cognitive_empathy,affective_empathy,affection,knowledge,shared_household,shared_household_years,ownership,ownership_count,expertise,expertise_years
count,40.000000,40.000000,40.000000,40.0,40.0000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,17.000000,40.000000,7.000000,40.000000,6.000000
mean,23.000000,0.450000,0.425000,1.0,1.7200,1.713889,2.181250,1.962500,2.156250,1.716944,2.100000,4.000000,3.175000,0.425000,8.382353,0.175000,1.142857,0.175000,2.083333
std,2.611611,0.503831,0.500641,0.0,0.5095,0.382332,0.515738,0.556287,0.657032,0.379819,0.470724,0.816497,0.812956,0.500641,7.114091,0.384808,0.377964,0.384808,1.114301
min,19.000000,0.000000,0.000000,1.0,1.0000,1.000000,1.000000,1.000000,1.250000,1.055556,1.166667,2.000000,2.000000,0.000000,0.500000,0.000000,1.000000,0.000000,0.500000
25%,21.000000,0.000000,0.000000,1.0,1.3000,1.444444,2.000000,1.750000,1.500000,1.458333,1.833333,4.000000,3.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.250000
50%,23.000000,0.000000,0.000000,1.0,1.6000,1.666667,2.250000,1.750000,2.125000,1.722222,2.000000,4.000000,3.000000,0.000000,8.000000,0.000000,1.000000,0.000000,2.500000
75%,25.000000,1.000000,1.000000,1.0,2.0000,1.916667,2.500000,2.250000,2.500000,2.044444,2.395833,5.000000,4.000000,1.000000,13.000000,0.000000,1.000000,0.000000,3.000000
max,28.000000,1.000000,1.000000,1.0,2.9000,2.444444,4.000000,3.250000,4.000000,2.422222,3.250000,5.000000,5.000000,1.000000,20.000000,1.000000,2.000000,1.000000,3.000000


In [22]:
# Sex
print("Number of female paricitpants =", np.sum(data['sex']==0))
print("Number of male paricitpants =", np.sum(data['sex']==1))
print("Percentage of woman =", np.sum(data['sex']==0)*100/40)

# dog expertise
print("Number of dog experts:", np.sum(data['dog_expert']==1))
print("Number of non-dog experts:", np.sum(data['dog_expert']==0))

Number of female paricitpants = 22
Number of male paricitpants = 18
Percentage of woman = 55.0
Number of dog experts: 17
Number of non-dog experts: 23


### Create a participants.tsv file

In [23]:
data.to_csv(work_dir / "participants.tsv", sep='\t',na_rep='n/a', index=False)

### Create a BIDS style dataset_description.json file 


In [24]:
import json

Now we can create a dictionary, where we store information on participants. 

In [28]:
dataset_dict = {
    "Name": "Action Observation and Action Task, Master Thesis",
    "Authors": ["Olaf Borghi", "Magdalena Boch"],
    "Funding": ["This project was supported by the Austrian Science Fund (FWF): W1262-B29 and by the Vienna Science and Technology Fund (WWTF), the City of Vienna and ithuba Capital AG through project CS18-012 , and the Messerli Foundation (S\u00f6renberg, Switzerland). The funders had no role in study design, data collection and analysis, decision to publish, or preparation of the manuscript."], 
    "BIDSVersion": "1.2.1", 
    "Date": "2021"
}

From the dictionary, we can now create a .json file named dataset_description.

In [29]:
# create the json file from the dict and store it in the wd
jsonString = json.dumps(dataset_dict,
                        sort_keys=False,
                        indent=4,
                        separators=(',', ': '))
jsonFile = open("dataset_description.json", "w")
jsonFile.write(jsonString)
jsonFile.close()

In [30]:
%%bash
cat dataset_description.json

{
    "Name": "Action Observation and Action Task, Master Thesis",
    "Authors": [
        "Olaf Borghi",
        "Magdalena Boch"
    ],
    "Funding": [
        "This project was supported by the Austrian Science Fund (FWF): W1262-B29 and by the Vienna Science and Technology Fund (WWTF), the City of Vienna and ithuba Capital AG through project CS18-012 , and the Messerli Foundation (S\u00f6renberg, Switzerland). The funders had no role in study design, data collection and analysis, decision to publish, or preparation of the manuscript."
    ],
    "BIDSVersion": "1.2.1",
    "Date": "2021"
}

### Create a BIDS style descriptives file (participants.json)

In [34]:
participant_dict = {
    "age": {
        "Description": "Age of the participant", 
        "Units": "years"
    }, 
    "sex": {
        "Description": "Sex of the participant", 
        "Levels": {
            "0": "Female", 
            "1": "Male",
            "2": "Other",
        }
    },
    "dog_expert": {
        "Description": "A person that either lived in a household with dogs, owned one or several dogs, or has other significant expertise (e.g., from training or working with dogs)",
        "Levels": {
            "0": "Non-expert",
            "1": "Expert"
        }
    },
    "handedness": {
        "Description": "Dominant hand of the participant", 
        "Levels": {
            "R": "right-handed"
        }
    }, 
    "perspective_taking": {
        "Description": "the ability to interpret situations from the perspective of others", 
        "Questionnaire": "Questionnaire of Cognitive and Affective Empathy (Reniers et al., 2016)",
        "Units": "arbitrary interval scale ranging from 1 to 4"
    }, 
    "online_simulation": {
        "Description": "the ability to put oneself in another person’s place to understand and anticipate their feelings", 
        "Questionnaire": "Questionnaire of Cognitive and Affective Empathy (Reniers et al., 2016)",
        "Units": "arbitrary interval scale ranging from 1 to 4"
    }, 
    "emotion_contagion": {
        "Description": "the tendency to automatically take over others’ emotions", 
        "Questionnaire": "Questionnaire of Cognitive and Affective Empathy (Reniers et al., 2016)",
        "Units": "arbitrary interval scale ranging from 1 to 4"
    }, 
    "proximal_responsivity": {
        "Description": "the affective responsivity when observing others’ emotions in a more close social context", 
        "Questionnaire": "Questionnaire of Cognitive and Affective Empathy (Reniers et al., 2016)",
        "Units": "arbitrary interval scale ranging from 1 to 4"
    }, 
    "peripheral_responsivity": {
        "Description": "the affective responsivity when observing others’ emotions in a more detached context, e.g., when watching characters in a movie", 
        "Questionnaire": "Questionnaire of Cognitive and Affective Empathy (Reniers et al., 2016)",
        "Units": "arbitrary interval scale ranging from 1 to 4"
    }, 
    "cognitive_empathy": {
        "Description": "the ability to construct a working model of the emotional states of others", 
        "Questionnaire": "Questionnaire of Cognitive and Affective Empathy (Reniers et al., 2016)",
        "Units": "arbitrary interval scale from 1 to 4"
    }, 
    "affective_empathy": {
        "Description": "the ability to be sensitive to and vicariously experience the feelings of others", 
        "Questionnaire": "Questionnaire of Cognitive and Affective Empathy (Reniers et al., 2016)",
        "Units": "arbitrary interval scale ranging from 1 to 4"
    }, 
    "affection": {
        "Description": "How much do you like dogs?", 
        "Levels": {
            "1": "Not at all", 
            "2": "A little", 
            "3": "Some", 
            "4": "Very much", 
            "5": "Extremely much"
        }
    }, 
    "knowledge": {
        "Description": "How much do you know about dog behaviour?", 
        "Levels": {
            "1": "Not at all", 
            "2": "A little", 
            "3": "Some", 
            "4": "Very much", 
            "5": "Extremely much"
        }
    }, 
    "shared_household": {
        "Description": "Have you ever shared or currently share a household with at least one dog?", 
        "Levels": {
            "1": "yes", 
            "0": "no"
        }
    }, 
    "shared_household_years": {
        "Description": "Time spent living with at least one dog in same household", 
        "Units": "years"
    }, 
    "ownership": {
        "Description": "Do you currently own a dog?", 
        "Levels": {
            "1": "yes", 
            "0": "no"
        }
    }, 
    "ownership_count": {
        "Description": "How many dogs do you currently own?", 
        "Units": "count"
    }, 
    "expertise": {
        "Description": "Do you have any experience in dog training or hobbies involving working with dogs (e.g. agility?)", 
        "Levels": {
            "1": "yes", 
            "0": "no"
        }
    }, 
    "expertise_years": {
        "Description": "If expertise was answered with yes: For how many years?", 
        "Unit": "years"
    }, 
    "expertise_open": {
        "Description": "if expertise was answered with yes: What kind of experience do you have?", 
        "Unit": "string"
    }
}

***add later to dict if I calculate FD:***

"NumberFD_run1": {"Description": "Scans exceeding framewise displacment (FD, i.e., scan-to-scan motion) threshold of 0.5 mm in run 1", "Units": "count"}, 

"PercentFD_run1": {"Description": "Scans exceeding framewise displacment (FD, i.e., scan-to-scan motion) threshold of 0.5 mm in run 1", "Units": "Percentage"}, 

"NumberFD_run2": {"Description": "Scans exceeding framewise displacment (FD, i.e., scan-to-scan motion) threshold of 0.5 mm in run 2", "Units": "count"}, 

"PercentFD_run2": {"Description": "Scans exceeding framewise displacment (FD, i.e., scan-to-scan motion) threshold of 0.5 mm in run 2", "Units": "Percentage"},

In [35]:
# create the json file from the dict and store it in the wd
jsonString = json.dumps(participant_dict,
                        sort_keys=False,
                        indent=4,
                        separators=(',', ': '))
jsonFile = open("participants.json", "w")
jsonFile.write(jsonString)
jsonFile.close()

In [36]:
%%bash
cat participants.json

{
    "age": {
        "Description": "Age of the participant",
        "Units": "years"
    },
    "sex": {
        "Description": "Sex of the participant",
        "Levels": {
            "0": "Female",
            "1": "Male",
            "2": "Other"
        }
    },
    "dog_expert": {
        "Description": "A person that either lived in a household with dogs, owned one or several dogs, or has other significant expertise (e.g., from training or working with dogs)",
        "Levels": {
            "0": "Non-expert",
            "1": "Expert"
        }
    },
    "handedness": {
        "Description": "Dominant hand of the participant",
        "Levels": {
            "R": "right-handed"
        }
    },
    "perspective_taking": {
        "Description": "the ability to interpret situations from the perspective of others",
        "Questionnaire": "Questionnaire of Cognitive and Affective Empathy (Reniers et al., 2016)",
        "Units": "arbitrary interval scale ranging from 1 to 

### Now we have a participants.tsv, a participant.json and a dataset_description.json file!

What is missing? A README file containing other relevant information.